# A practical introduction to Value Function Iteration (VFI)

This Julia notebook is intended to serve as an introduction to using VFI to solve recursive dynamic economic models. I assume that you already know the deterministic neoclassical growth model and have been exposed to dynamic programming, so today's focus will solely be on writing code to solve it. Throughout my exposition, I will try to explain as many `non-obvious' lines of code or choices as possible, with a heavy emphasis on what practical issue they address, especially when the problem we are solving stops becoming well-behaved like this one.

# 1. Initialization

We first need to load all the packages and initialize all the objects that we will be using throughout the rest of the code. This includes our parameters (both economic and numerical) and functions.

In [ ]:
using Interpolations
using Optim
using LinearAlgebra
using Plots

const β = 0.96
const α = 0.4

f(k) = k^α
u(c) = c ≥ 0.0 ? log(c) : -1e10

u (generic function with 1 method)

There are two noteworthy matters. 
1. I am using these functional forms and parameter values (note that $\delta = 1$ implicitly) because this parameterization lends itself to an analytical solution from Sargeant Ljundvist with which we can verify our numerical solution against. 
$$
v(k) = \frac{\ln(1-\alpha\beta)}{1-\beta}
$$
Of course, the moment we depart from this parameterization, our analytical benchmark disappears (I will explain another way to verify that our numerical solution is correct in the next notebook). However, I still find this setting to be extremely useful when learning a new algorithm or computational tool (even if it is complete overkill for solving this model) because having that analytical benchmark is useful for confirming whether I am implementing the extension correctly or not. 

2. For expository purposes I am declaring the parameters in an interpreted way, i.e. everything is declared line-by-line. For interpreted languages like MATLAB and Python, this is what we would do anyway. However, Julia is a compiled language, which means that the overwhelming majority of our code should be written in a function or housed inside a struct. In settings where performance becomes crucial, this would usually be terrible practice. (Declaring our parameters to be a constant is not strictly necessary. However, it is a minor optimization because it tells Julia's compiler that this parameter will not change.)

## 1.1 Utility function

You may be wondering why the utility function is written this way. In Julia, this is called the *tenery operator* and it reads as the following: "evaluate the statement to the left of the ?. If the statement (consumption is weakly positive) is true, then return what is to the left of the colon ($log(c)$). If it's false, return what is to the right of the colon (an arbitrary, but very negative number). If tenery operators are not available in your programming language, a custom function with an if-statement should work as well.

In practice, this is very useful over simply writing $u(c) = log(c)$ for two reasons. 
1. Economics tells us that utility functions are only defined over weakly positive consumption, yet there is nothing mathematical that inherently forces this limitation for most utility functions. This becomes really important in the optimization step later, because when we are trying to maximize the utility function, we need to eliminate solutions that are only possible with negative consumption. By returning a very large negative number, we can hopefully steer the optimizer away from choices that imply negative consumption. The optimizer may still attempt it, but it will be attached with such a large penalty such that these choices will never appear as the solution.
2. In the rare case that mathematics enforces this limitation (e.g. the log-function), our code would usually stop entirely, and return an error message telling us that taking the log of a negative number is not possible without complex numbers. However, since our custom function is still defined over negative numbers, we bypass this issue entirely.

## 1.2 Creating the state-space (grid)

Next, we create our state-space. Mathematically our state-space is continuous with infinitely-many elements. However, this is impossible to store on our computer, not to mention perform complicated calculations on. In practice, we substitute this with a discrete grid of points (with finitely many elements) which in this case, is a 101-element vector of points for capital from 0.0 to 90.0.

### Aside - interpolation vs discrete state space (DSS)
Most introductions to VFI are done with a purely discretized state-space with no interpolation at all. The main difference that this results in is that the later maximization step gets replaced with a grid-search. I.e. instead of using optimizers on a continuous function, we instead evaluate the objective function along a finite number of possible choices and pick the choice that results in the largest number. There are two major benefits to the DSS approach:
1. Simplicity - finidng the maximum among a row with finitely-many elements is much simpler (and possibly faster) than invoking optimizers. 
2. Stability - a significant source of instability for interpolation is that algorithms usually want to engage in **extrapolation**, especially at the boundaries of the grid. However, extrapolation can result in extreme instability since there are no economic conditions to impose in the extrapolated region by construction. There are ways to side-step this, but they introduce risks.

However, there are also some major downsides to this approach (and ultimately why I prefer to teach with interpolation)
1. Speed/accuracy - holding the desired level of accuracy constant, DSS requires far more gridpoints for the result of the maximization step to not be the result of **rounding error**. Suppose that the true optimal choice for $k^\prime$ at $k = 20.0$ is $k^\prime = 15.4$, but your grid is only defined on $...15.0, 16.0, ...$. Purely discretizing the state space will result in $k^\prime = 15.0$ being chosen, as it is the local maxima among a finite set of points - the value function isn't defined at $15.4$! You would need to increase the fineness of your grid to avoid that $0.4$ rounding error, thereby increasing computational cost. Meanwhile, interpolating between gridpoints results in the value function being defined at 15.4, even if your gridpoints are defined on $...10.0, 20.0...$. (This doesn't mean that there aren't any approximation errors though! E.g. your choice of basis functions might result in 15.35 being chosen as the optimum.)
2. Speed - VFI is very slow, especially if your state-space is high dimensional. This is irrespective of whether you interpolate your desired functions or not. I will go over algorithms that are MUCH faster (and more accurate) in future notebooks, but they only work with fully continuous functions, which DSS does not permit.

In [ ]:
const kmin = 0.0;
const kmax = 90.0;
const nk = 101;
kgrid = range(kmin, kmax, length=nk)

0.001:0.0900890890890891:90.0

There are three extremely important numerical issues to consider.
### 1.2.1 How many grid-points we should use?

There is a trade-off when deciding how many gridpoints to use. If *too few* gridpoints are used, then it becomes difficult to find an accurate solution (especially if the state space is completely discretized - the results of the optimization step would more be due to rounding error rather than being the true optimizer). However, we wouldn't want to use *too many* gridpoints either, because it would dramatically increase the time (and memory) needed for each iteration. In our one-dimensional model today, this would not be an issue on modern hardware, but since the number of gridpoints increases *exponentially* with the dimensionality of the state-space, we also need to be wary against too many gridpoints. 

Ex-ante, it is impossible to know what the optimal number of gridpoints is, especially since it is heavily model and algorithm-dependent. One useful rule is to start with a lower estimate of the number of gridpoints in one run, then slowly increase it. If the solution changes significantly, then it was not enough points. Only when your solution is robust to this parameter, can you be confident that it is enough.

### 1.2.2 How wide should our bounds be?

Again, there is a trade-off associated with this choice. We cannot make it *too narrow*. At best, we would not accurately capture the model's dynamics while at worst, we would not find convergence at all. We also cannot make it *too wide*, because for the same number of gridpoints, the space between them would increase, and the accuracy of our solution would suffer (unless more gridpoints are used, thus increasing computatio time). 

Fortunately, our problem is sometimes blessed with natural limits. For example, negative capital is not possible, so setting the lower bound to zero or close to zero is a good choice. Here, the upper-bound is less obvious. Sometimes, economics can give guidance. For instance, in this problem where a unique steady-state exists, any grid that does not include the steady-state value of capital would likely result in a terrible approximation. On the flip-side, the upper-bound likely wouldn't need to be significantly larger than this. This is a parameter worth playing around (there are literatures where the numerical results are **highly** sensitive to this), but like before, you can be confident that your grid is wide enough when enlarging it stops resulting in the results changing.

### 1.2.3 How should we space our points out across the bounds?

In this code (and most introductory code), the gridpoints for capital are uniformly spread out. In low dimensional and well-behaved environments like this, this is perfectly fine since increasing the number of grid-points is not likely to be overly expensive. However, this is not **optimal**, and if your state-space is high dimensional (in practice, 3 dimensions) or your desired functions are nasty (e.g. it has kinks, discontinuities, or regions of high curvature), equidistant-spaced grids become outright prohibitive. For a given finite budget of gridpoints, they should be concentrated around regions where the function is badly behaved because more gridpoints are needed to capture this behavior and increase your solutions' accuracy.

## 2. Writing the Bellman operator

Below is the function that implements the Bellman operator for a given guess of the value function. First, it finds the argument $k^\prime$, bounded between kmin and $f(k)$ that attains the maximum of the Bellman equation at a given point in the discretized state-space. The maximized value and accompanying maximizer are then stored in the appropriate vector which was already pre-declared. Finally, it outputs the new value function, or $T(v)$.

Since the Optim.jl's optimize function is used to find the **minimum** of functions, we transform our problem by taking the negative of everything so we are actually finding the **maximum**.

This is arguably the most important part of the code, so make sure you understand it!

In [ ]:
function bellman_operator(v_old)
    value_func = LinearInterpolation(kgrid, v_old)
    Tv_new = zeros(length(grid))
    
    for (i,k) in collect(enumerate(grid))
        results = optimize(k′ -> -u(f(k) - k′) - β * value_func([k′]), kmin, f(k))
        Tv_new[i] = -results.minimum
        policy[i] = results.minimizer
    end
    return Tv_new
end

bellman_operator (generic function with 1 method)

There are two noteworthy matters. 

1. While we are inputting a vector (our previous guess of the value function evaluated at our **finite** grid of points), we are ultimately maximizing over the continuous function **value_func**, which is constructed via linear interpolation on our inputted guess. This means that we are   

2. A critical issue is the bounds that we feed to the optimization algorithm. Economics tells us that as per the resource constraint, we the largest that $k^\prime$ can be is $f(k)$. Once again, there is nothing that inherently forces the optimizer to respect this limitation unless we force it to. This may become a very severe problem in the initial few iterations where our approximation of $v(k)$ is poor, a

In principle, this should not be an issue because based on the way the utility function is written, a massive penalty should be outputted for choices of $k^\prime$ that require negative consumption and indeed, for standard interpolation schemes like linear and cubic splines. However, when as we use more advanced tools, implementing these kinds of checks and constraints become increasingly crucial, especially during the initial few iterations where likely poor approximation of $v(k)$ can lead to non-sensical optimums.

## 2. Main loop

Below is the iterative loop. Once you understand the Bellman operator, the rest of this code should be fairly simple. All the loop does is:
1. Input a guess of the value function (v_old)
2. Apply the Bellman operator to it, outputting a new value function (v_new) 
3. Calculate the distance between the guess and new value function
4. If the distance is small enough (less than tol) then declare convergence and end the loop. Otherwise, update v_old in the next iteration to be the current iteration's v_new, and repeat.

In [ ]:
iiter = 0;
max_iter = 600;
tol = 1e-6;

v_old = u.(f.(grid))

while iiter < max_iter
    iiter += 1

    v_new = bellman_operator(v_old)

    diff = norm(v_new - v_old, Inf)

    println("Iteration: $iiter, Error: $diff")
    if diff < tol
        break
    end 
    
    v_old = copy(v_new)
end

## 2.1. How small is close `enough'? 

In other words, what should we set the numerical tolerance parameter to? Once again, there's no single definitive number. On modern hardware (which uses 64-bit floating point representation), the absolute maximum you should set your tolerance parameter to is 10^-16, or machine epsilon, which roughly speaking is the smallest distance between two numbers that a computer can reliably distinguish. I.e. if two numbers are within machine epsilon distance of each other, the computer may think they are equivalent numbers.

However, if I had to give a rule of thumb, then it is somewhere between 10^-5 to 10^-10.
